## Loading black-box functions with TT-Cross

TT-Cross, also known as tensor cross-interpolation (TCI), is a method to load black-box functions in tensor trains or MPS. This library implements several variants of TT-Cross, each with its strengths and weaknesses. In this example, we show the usage of the DMRG-based variant `cross_DMRG`, which is specially suitable for MPS with small physical dimension. However, these examples apply analogously to all other variants by just replacing the method.

The black-box function given as input corresponds to a `BlackBox` object. There are several subclasses corresponding to different input black-boxes, applicable in different scenarios. In this notebook, we show a brief example for each type.

In [19]:
import numpy as np
import matplotlib.pyplot as plt

from seemps.analysis.mesh import Mesh, RegularInterval
from seemps.analysis.factories import mps_interval
from seemps.analysis.cross import cross_dmrg
import seemps.tools

seemps.tools.DEBUG = 2

### 1. `BlackBoxLoadMPS`

This black-box representation allows to load functions in a MPS by imposing a quantization on its degrees of freedom with a given base, or *physical dimension*. This enables to impose different tensor orders, such as *serial (A)* and *interleaved (B)*. In this example we load the function

$$
f(x, y) = e^{-(x^2 + y^2)}
$$

on a binary MPS with standard physical dimension 2 and serial order.

The function `func` must act on the whole input tensor, following the convention that its first index labels each degree of freedom or dimension.

In [20]:
from seemps.analysis.cross import BlackBoxLoadMPS

func = lambda tensor: np.exp(-np.sum(tensor**2, axis=0))

start, stop = -1, 1
num_qubits = 10
interval = RegularInterval(start, stop, 2**num_qubits)
dimension = 2
mesh = Mesh([interval] * dimension)

black_box = BlackBoxLoadMPS(func, mesh, base=2, mps_order="A")
mps = cross_dmrg(black_box).mps

 Cross sweep   1 with error(1000 samples in norm-inf)=0.08093730903765317, maxbond=2, evals(cumulative)=144
 Cross sweep   1 with error(1000 samples in norm-inf)=1.4290195931743188e-05, maxbond=4, evals(cumulative)=624
 Cross sweep   2 with error(1000 samples in norm-inf)=4.7628567756419216e-14, maxbond=8, evals(cumulative)=1920
 State converged within tolerance 1e-12


### 2. `BlackBoxLoadTT`

This black-box representation allows to load functions in a tensor-train with no such quantization, by assigning a full tensor to each function variable. 
Even though we use it here, `cross_dmrg` is not optimal for this structure and it largely overestimates the bond dimension. Instead, its better to use `cross_maxvol` or `cross_greedy`.

In [21]:
from seemps.analysis.cross import BlackBoxLoadTT

func = lambda tensor: np.exp(-np.sum(tensor**2, axis=0))

start, stop = -1, 1
num_nodes = 1000
interval = RegularInterval(start, stop, num_nodes)
dimension = 2
mesh = Mesh([interval] * dimension)

black_box = BlackBoxLoadTT(func, mesh)
mps = cross_dmrg(black_box).mps

 Cross sweep   1 with error(1000 samples in norm-inf)=1.304512053934559e-14, maxbond=993, evals(cumulative)=1000000
 State converged within tolerance 1e-12


### 3. `BlackBoxLoadMPO`

This black-box representation allows to load a bivariate function in a MPO, by loading its equivalent MPS and unfolding it at the end. In this example, we load in MPO the bivariate function

$$
f(x, y) = e^{-(x^2 + y^2)}.
$$

The function `func` must act on two input values labeling the rows and columns of the MPO respectively.

In [18]:
from seemps.analysis.cross import BlackBoxLoadMPO
from seemps.truncate.simplify_mpo import mps_as_mpo

func = lambda x, y: np.exp(-(x**2 + y**2))

start, stop = -1, 1
num_qubits = 10
interval = RegularInterval(start, stop, 2**num_qubits)
dimension = 2
mesh = Mesh([interval] * dimension)

black_box = BlackBoxLoadMPO(func, mesh)
mps = cross_dmrg(black_box).mps
mpo = mps_as_mpo(mps)

 Cross sweep   1 with error(1000 samples in norm-inf)=0.09425112330182073, maxbond=4, evals(cumulative)=528
 Cross sweep   1 with error(1000 samples in norm-inf)=2.4101306458801375e-05, maxbond=16, evals(cumulative)=6992
 Cross sweep   2 with error(1000 samples in norm-inf)=2.537969834293108e-13, maxbond=59, evals(cumulative)=71216
 State converged within tolerance 1e-12


### 4. `BlackBoxComposeMPS`

This black-box representation allows composing scalar functions on several MPS, given by `mps_list`. In this example we compose the function

$$
f(x,y,z) = x\cdot \sin(y z) + y \cdot \cos(x z)
$$

on three initial MPS representing $x$, $y$ and $z$. The function `func` must act on the whole list of MPS.

In [12]:
from seemps.analysis.cross import BlackBoxComposeMPS

func = lambda mps: mps[0] * np.sin(mps[1] * mps[2]) + mps[1] * np.cos(mps[0] * mps[2])

start, stop, num_qubits = -1, 1, 10
interval = RegularInterval(start, stop, 2**num_qubits)
mps_x = mps_interval(interval)

black_box = BlackBoxComposeMPS(func, [mps_x, mps_x, mps_x])
mps = cross_dmrg(black_box).mps

 Cross sweep   1 with error(1000 samples in norm-inf)=0.11293525051035298, maxbond=2, evals(cumulative)=68
 Cross sweep   1 with error(1000 samples in norm-inf)=1.723676025844334e-05, maxbond=4, evals(cumulative)=300
 Cross sweep   2 with error(1000 samples in norm-inf)=1.354472090042691e-14, maxbond=8, evals(cumulative)=972
 State converged within tolerance 1e-12
